In [1]:
import numpy as np
import mesh_to_sdf
import trimesh
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import tensorflow as tf
import mcubes
import open3d as o3d
from Subdivision.Datastructures import Octree
import yaml

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("../3DModels/monkj.obj")
bounds = mesh.get_axis_aligned_bounding_box()
extent = np.max(bounds.get_extent(), axis=-1)
mesh.scale(1. / extent, center=bounds.get_center())

TriangleMesh with 3229 points and 1149 triangles.

In [3]:
verts = np.array(mesh.vertices)
print(verts.min(), verts.max())

-0.49999999999999994 0.49999999999999994


In [4]:
N = 250000
pcd = mesh.sample_points_uniformly(N)
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
np.array(pcd.points).max()
#o3d.visualization.draw_geometries([pcd])

0.49958255840195004

In [10]:
def subdivide(points, depth):
    return True

def drop(points, depth):
    if depth <= 3 and points.shape[0] < 1000:
        return True
    return False

# points = np.array([[-0.25, -0.25, -0.25],
#                    [-0.25, -0.25, 0.25],
#                    [-0.25, 0.25, -0.25],
#                    [-0.25, 0.25, 0.25],
#                    [0.25, -0.25, -0.25],
#                    [0.25, -0.25, 0.25],
#                    [0.25, 0.25, -0.25],
#                    [0.365, 0.365, 0.365],
#                    [0.365, 0.365, 0.24],
#                    [0.365, 0.24, 0.365],
#                    [0.365, 0.24, 0.24],
#                    [0.24, 0.365, 0.365],
#                    [0.24, 0.365, 0.24],
#                    [0.24, 0.24, 0.365],
#                    [0.24, 0.24, 0.24],])

points = np.random.random((300, 3)) - 0.5
print(points.min(), points.max())

octree = Octree(origin = np.array([-.5, -.5, -.5]),
                extents = np.array([1., 1., 1.]),
                points = np.array(pcd.points),
                max_depth = 3,
                subdivide = subdivide,
                drop = drop)
octree.visualize(show_boxes=True, show_coordinate_frames=False, show_points=True)
#print(octree.root_node.get_child_bounding_box(0))
#print(octree.get_debug_string())
print(octree.check_integrity())
debug_dict = octree.get_debug_dict()
with open("octree_monkj.yaml", 'w+') as f:
    yaml.dump(debug_dict, f)


-0.4990366795288532 0.49929511679680794
True


In [6]:
octree = o3d.geometry.Octree(max_depth=3)
octree.convert_from_point_cloud(pcd, size_expand=0.01)
box = o3d.geometry.TriangleMesh.create_coordinate_frame()
#box.paint_uniform_color(np.array([1,0,0]))
o3d.visualization.draw_geometries([box])

In [21]:
bounds = []
def f(n, ni, b=bounds):
    if ni.depth == 3:
        b.append(ni.origin)
octree.traverse(f)

In [22]:
bounds = np.array(bounds)
bounds.shape
bounds_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(bounds))

In [8]:
points = np.array([[0.25, -0.25, 0.25],
                   [-0.25, -0.25, -0.25],
                   [0.25, -0.25, -0.25]])

origin = np.array([-.5, -.5, -.5])
extents = np.array([1., 1., 1.])
index = np.zeros(points.shape[0]).astype('int8')
mask = np.array(points <= (origin + extents / 2.)).astype('int8')

for i in range(mask.shape[1]):
    index = 2 * index + mask[:,i]
index

array([2, 7, 3], dtype=int8)

In [11]:
child_extents = extents / 2.
child_origin = np.array([origin[i] + (0 if ((index[0] >> i) & 1) == 1 else child_extents[i]) for i in range(points.shape[1]-1, -1, -1)])

In [14]:
child_origin

array([ 0. , -0.5,  0. ])